necessary videos

In [ ]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt

print(tf. __version__)

In [ ]:
# from calendar import EPOCH


IMAGE_SIZE = 256
BATCH_SIZE = 32   #Standard batch size
CHANNELS = 3
EPOCHS = 50


In [ ]:
# Specify directory first
# one call will load all the images into tensorflow


In [ ]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "PlantVillage", #directory
    shuffle=True,   # To randomly shuffle the images.
    image_size = (IMAGE_SIZE, IMAGE_SIZE),
    batch_size = BATCH_SIZE 
)

In [ ]:
class_names = dataset.class_names
class_names

In [ ]:
# 68 * 32
len(dataset)

In [ ]:
#explore images.
for image_batch, label_batch in dataset.take(1):
    # print(image_batch.shape)
    # print(image_batch[0])
    # print(image_batch[0].numpy())  #convert it into numpy
    print(image_batch[0].shape)

    # Every element that you get is a tensor,so you need to
    #  convert it to numpu
    # print(label_batch.numpy())

Visualize images.

In [ ]:
for image_batch, label_batch in dataset.take(1):
    # plt.imshow(image_batch[0].numpy().astype("uint8"))
    plt.imshow(image_batch[0].numpy().astype("uint8"))


    # labelling the title
    plt.title(class_names[label_batch[0]])
    # Remove the axis labeling
    plt.axis("off")

Displaying a many images once

In [ ]:
#Increase the dimension
plt.figure(figsize=(15, 15))
for image_batch, label_batch in dataset.take(1):
    
    for i in range(12):
        ax = plt.subplot(3, 4, i +1)
        plt.imshow(image_batch[i].numpy().astype("uint8"))

        # labelling the title
        plt.title(class_names[label_batch[0]])
        # Remove the axis labeling
        plt.axis("off")

Spliting our dataset into training and testing

In [ ]:
len(dataset)

In [ ]:
#  80% ==> training
#  20% ==> 10% validation, 10 % test

In [ ]:
train_size = 0.8
len(dataset) * train_size

I will take 54

In [ ]:
train_ds = dataset.take(54)
len(train_ds)

The ramaining will  be for test

In [ ]:
test_ds = dataset.skip(54)
len(test_ds)

I need 10% for validation

In [ ]:
val_size = 0.1
len(dataset) *val_size

In [ ]:
# Taking 10 % from training dataset
val_ds = test_ds.take(6)
len(val_ds)

In [ ]:
# The actual testind dataset. 
test_ds = test_ds.skip(6)
len(test_ds)

Creating a function

In [ ]:


def get_dataset_partitions_tf(ds, train_split=0.8,val_split=0.1, test_split=0.1, shuffle=True,shuffle_size=10000):

    ds_size = len(ds)

    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)

    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)

    train_ds = ds.take(train_size)

    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)




    return train_ds, val_ds, test_ds

In [ ]:
# Calling the function above
train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset)


In [ ]:
print(f"Length of train dataset is : {len(train_ds)}")
print(f"Length of validation dataset is : {len(val_ds)}")
print(f"Length of test dataset is : {len(test_ds)}")


caching. In data science, repetitions of the same routines could be cached to speed the experiments or decrease data access latency. The most straightforward adaptation policy is to take advantage of the locality of reference principle.

In [ ]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)



Preprocessing

In [ ]:
resize_and_rescale = tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(IMAGE_SIZE, IMAGE_SIZE),
    # Preprocessing API
    layers.experimental. preprocessing.Rescaling(1.0/255)
    ])

Argumentation

In [ ]:
data_augumentation = tf.keras.Sequential(
    [layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(0.2),
    
    ]
)

Creating the Model

In [ ]:
input_shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
n_classes = 3

model = models.Sequential([
    # resize_and_rescale,
    data_augumentation,
    layers.Conv2D(32,(3, 3), activation='relu', input_shape = input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, kernel_size = (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64,(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64,(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64,(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes, activation ='softmax'),


]
    
)
model.build(input_shape=input_shape)

In [ ]:
model.summary()

In [ ]:
model.compile(
    optimizer='adam',
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

Training the network


In [28]:
# from tabnanny import verbose


history = model.fit(
    train_ds,
    epochs= EPOCHS,
    batch_size = BATCH_SIZE,
    verbose=1,
    validation_data = val_ds
)

54/54 [==============================] - 197s 3s/step - loss: 3.1446 - accuracy: 0.6262 - val_loss: 0.6370 - val_accuracy: 0.7656
Epoch 2/50
52/54 [===========================>..] - ETA: 6s - loss: 0.4878 - accuracy: 0.8299